**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install lightgbm
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from IPython.display import display, Markdown

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Loading and Preprocessing the Data -- { display-mode: "form" }
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']

output = "Survived"

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values.reshape(-1)

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values.reshape(-1)

## Heterogénne komisie

V predošlom notebook-u sme sa venovali metódam na tréning homogénnych komisií: t.j. komisií zložených z niekoľkých modelov toho istého druhu (v našom prípade rozhodovacích stromov). Tieto metódy boli schopné modely vytvoriť a natrénovať automaticky.

V tomto notebook-u sa budeme venovať heterogénnym komisiám, ktoré si vyžadujú trochu viac práce: každý model treba vytvoriť osobitne a potom ich spojiť do komisie pomocou nejakého metaklasifikátora ako je napr. `sklearn.ensemble.VotingClassifier` (prípadne `sklearn.ensemble.VotingRegressor` pre regresiu). Odmenou za túto prácu navyše by však mali byť lepšie výsledky: heterogénne modely robia typicky veľmi odlišné chyby, takže heterogénna komisia vie často dosiahnuť lepšie zovšeobecnenie než homogénna.

### Komisia pomocou `VotingClassifier`

Teraz v krátkosti ilustrujeme ako na našu úlohu aplikovať triedu `VotingClassifier`. Začneme vytvorením zoznamu modelov, z ktorých chceme vytvoriť komisiu. Najprv môžeme vytvoriť aj každý z nich osobitne a pomocou krížovej validácie im trochu vyladiť hyperparametre, podobne ako sme to robili v predošlých notebook-och.



In [ ]:
def crossval(model):
    scores = cross_validate(model, X_train, Y_train, cv=10, return_train_score=True)
    display(Markdown("train: {:.5f}; **test: {:.5f}**".format(
        scores['train_score'].mean(),
        scores['test_score'].mean()
    )))

---
#### Úloha 1: Ladenie hyperparamov pre každý klasifikátor samostatne

**V bunkách nižšie skúste experimentovať s hyperparametrami klasifikátorov a nájdite nastavenie, ktoré obstojne funguje v rámci krížovej validácie.**  Pomôcka: Ak si spustíte ?NameOfTheClassifier, zobrazí sa vám dokumentačný reťazec klasifikátora

---


In [ ]:
dtree_model = DecisionTreeClassifier(
    
    # ---
    
)
crossval(dtree_model)

In [ ]:
lgbm_model = LGBMClassifier(
    
    # ---
    
)
crossval(lgbm_model)

In [ ]:
knn_model = KNeighborsClassifier(
    
    # ---
    
)
crossval(knn_model)

In [ ]:
svc_model = svm.SVC(
    
    # ---
    
)
crossval(svc_model)

In [ ]:
logistic_model = LogisticRegression(
    
    # ---
    
)
crossval(logistic_model)

In [ ]:
estimators = [
    ("dtree", dtree_model),
    ("lgbm", lgbm_model),
    ("knn", knn_model),
    ('svc', svc_model),
    ('logistic', logistic_model)
]

Zoznam pošleme ako argument konštruktoru triedy `VotingClassifier`. Špecifikovať môžeme aj ďalšie parametre, napr. režim hlasovania – opis jednotlivých argumentov je možné nájsť v dokumentácii. Po skonštruovaní klasifikátor natrénujeme. Tým sa na dátach natrénujú všetky čiastkové modely.



In [ ]:
model = VotingClassifier(estimators)
crossval(model)

### Komisia pomocou `StackingClassifier`

Ako ďalšiu alternatívu môžeme namiesto hlasovania použiť stacking. V tom prípade najprv natrénujeme skupinu modelov a potom ich výstupy pridáme do dátovej množiny ako ďalšie stĺpce. Na vrch nakoniec umiestnime ďalší klasifikátor – t. j. natrénujeme ho na celej dátovej množine vrátane nových stĺpcov.

Tento model 2. úrovne môže využívať predpovede modelov 1. úrovne, napr. sa môže naučiť, ktoré modely by vedeli najlepšie predikovať pre tento druh vzorky a podľa toho ováhovať ich predikcie a pod.

Vytvorme si teda `StackingClassifier` s našou skupinou modelov 1. úrovne a ako model 2. úrovne použime logistickú regresiu.



In [ ]:
model = StackingClassifier(
    estimators,
    final_estimator=LogisticRegression(C=10),
    cv=10
)
crossval(model)

### Testovanie

Teraz vezmite najlepšiu komisiu a otestujte ju na testovacej množine. Pri troche šťastia by mala dosiahnuť lepšie výsledky než ktorýkoľvek jednotlivý model.



In [ ]:


# ---


accuracy_score(Y_test, y_test)